In [ ]:

from selenium.webdriver import Chrome

In [ ]:
from selenium.webdriver.chrome.service import Service

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
import cv2

In [ ]:
from numpy import uint8,frombuffer

In [ ]:
from base64 import b64decode

In [ ]:
from requests import get

In [ ]:
import os

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
end = "/html/body/div[2]/c-wiz/div[3]/div[1]/div/div/div/div/div[1]/div[2]/div[1]/div[2]/div[1]/div"

In [ ]:
driver = Chrome(service=Service(ChromeDriverManager().install()))
actions = ActionChains(driver)

In [ ]:
queries = []

with open("queries.txt") as file:
    queries.extend(file.read().split("\n"))
    file.close()

In [ ]:
queries

In [ ]:
def createDir(query):
    folder_path = os.path.join(os.path.abspath("images"),query)
    print(folder_path)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path)
        print("Dir Created")
    
    return folder_path

def getContent(source):
    if source.startswith("data:image"):
        encoded = source.split(",")[-1]
        content = b64decode(encoded)
        extension = source.lstrip("data:image/").split(";")[0]
    
    else:
        response = get(source)
        content = response.content
        head = response.headers
        extension = head["Content-Type"].lstrip("image/")
    
    return extension,content

def scroll():
    endTag = driver.find_element(By.XPATH,end)
    show_more = driver.find_element(By.CLASS_NAME,"LZ4I")
    # anyway = driver.find_element(By.CLASS_NAME,"XfJHbe")
    
    while not endTag.is_displayed():
        driver.execute_script("window.scrollBy(0,100)")

        if show_more.is_displayed():
            show_more.click()
            print("Show more clicked")

In [ ]:
for query in queries:
    cv2.destroyAllWindows()

    count = 1

    folder = createDir(query)

    url = "https://www.google.com/images?q=" + query
    driver.get(url)
    
    scroll()

    image_elements = driver.find_element(By.CLASS_NAME,"mJxzWe").find_elements(By.TAG_NAME,"img")
    
    for imgTag in image_elements:
        
        actions.move_to_element(imgTag).perform()
        link = imgTag.get_attribute("src")
        cls_name = imgTag.get_attribute("class")

        if cls_name != "rg_i Q4LuWd" : continue

        extension,content = getContent(link)

        file_name = os.path.join(folder,f"{str(count)}.{extension}")
        with open(file_name,"wb") as file:
            count += 1
            file.write(content)
            file.close()

        img = cv2.imdecode(frombuffer(content,uint8),cv2.IMREAD_COLOR)
        cv2.imshow("Frame",img) # cv2.resize(img,(500,500),cv2.INTER_))
        cv2.waitKey(1000)

cv2.destroyAllWindows()
    

In [ ]:
cv2.destroyAllWindows()

In [ ]:
driver.close()
driver.quit()